# Exercise 1.2
## This *.ipynb file contains:



In [2]:
import numpy as np
import torch
from Exercise1_1 import LQRSolver

In [3]:
H = torch.tensor([[1.0, 2.0], [-2.0, -3.0]], dtype=torch.double)
M = torch.tensor([[1.0,0.0], [0.0,1.0]], dtype=torch.double)
sigma = torch.tensor([[[0.5249],[0.4072]]], dtype=torch.double) 
C = torch.tensor([[2.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite
D = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive definite
R = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite
T = torch.tensor(1.0, dtype=torch.double)
method = 'rk4'

In [6]:
solver = LQRSolver(H, M, sigma, C, D, R, T, method)

In [52]:
N = 5000
t0 = torch.tensor(0.1,dtype = torch.double)

In [54]:
#tau = torch.clone((T-t0)/N)

In [56]:
time_grid_for_MC = torch.linspace(t0,T,N,dtype = torch.double)

In [58]:
X0 = 0.5*torch.ones([1,1,2], dtype=torch.double)
dt_for_MC = time_grid_for_MC[1:]-time_grid_for_MC[:-1]
S = solver.solve_riccati_ode(time_grid_for_MC.unsqueeze(0)).squeeze()
multpX = - M@torch.linalg.inv(D)@M.T
multpalp = -torch.linalg.inv(D)@M.T@S

In [59]:
print(solver.value_function(t0.unsqueeze(0),X0))

tensor(0.8088, dtype=torch.float64)


In [60]:
_int = torch.einsum('bii->b',sigma.squeeze(1)@sigma.squeeze(0).T@S)

value_for_varif = (X0@S[0]@X0.transpose(1,2) + torch.tensor(0.5)*dt_for_MC@(_int[1:]+_int[:-1]).unsqueeze(1)).squeeze()

print(value_for_varif)

tensor(0.8088, dtype=torch.float64)


In [62]:
J_list = []

for iii in range(500):
    
    J_sample = []
    
    for ii in range(10):
        
        X_0_N = X0
        
        for i in range(N-1):
            
            X_next = ((torch.eye(2)+ dt_for_MC[i]*(H + multpX@S[i]))@ X_0_N[i:].transpose(1,2)+sigma*torch.sqrt(dt_for_MC[i])*torch.randn(1)).transpose(1,2)
            X_0_N = torch.cat((X_0_N, X_next), dim=0)

        alp = multpalp@X_0_N.transpose(1,2)
        int_ = alp.transpose(1,2)@D@alp + X_0_N@C@X_0_N.transpose(1,2)
        J = X_0_N[-1]@R@X_0_N[-1].T + torch.tensor(0.5)*dt_for_MC@((int_.squeeze(1)[1:]+int_.squeeze(1)[:-1]))

        J_sample.append(J)
        
    J_sample_tensor = torch.stack(J_sample)
    J_list.append(torch.mean(J_sample_tensor).unsqueeze(0))
    
J_list_tensor = torch.cat(J_list, dim=0)

In [65]:
print(torch.sort(J_list_tensor)[0][:20])

tensor([0.7140, 0.7846, 0.8294, 0.8432, 0.8455, 0.8545, 0.8586, 0.8634, 0.8641,
        0.8712, 0.8901, 0.9044, 0.9093, 0.9103, 0.9243, 0.9322, 0.9402, 0.9491,
        0.9565, 0.9657], dtype=torch.float64)


In [ ]:
#Multiprocessing Version
from torch.multiprocessing import Pool, set_start_method
def worker(iteration, params):
    
    C = params['C']
    D = params['D']
    R = params['R']
    X0 = params['X0']
    H = params['H']
    dt = params['dt']
    multX = params['multX']
    multa = params['multa']
    sig = params['sig']
    
    J_sample = []
    
    for ii in range(10):
        
        X_0_N = X0
        
        for i in range(N-1):
            
            X_next = ((torch.eye(2)+ dt[i]*(H + multX@S[i]))@ X_0_N[i:].transpose(1,2)+sig*torch.sqrt(dt[i])*torch.randn(1)).transpose(1,2)
            X_0_N = torch.cat((X_0_N, X_next), dim=0)

        alp = multa@X_0_N.transpose(1,2)
        int_ = alp.transpose(1,2)@D@alp + X_0_N@C@X_0_N.transpose(1,2)
        J = X_0_N[-1]@R@X_0_N[-1].T + torch.tensor(0.5)*dt@((int_.squeeze(1)[1:]+int_.squeeze(1)[:-1]))

        J_sample.append(J)
        
    J_sample = torch.stack(J_sample)


    return torch.mean(J_sample).unsqueeze(0)

if __name__ == '__main__':
    
    try:
        set_start_method('spawn')  # 对于PyTorch，'spawn'或'forkserver'启动方法通常比较稳定
    except RuntimeError:
        pass
    
    H = torch.tensor([[1.0, 2.0], [-2.0, -3.0]], dtype=torch.double)
    M = torch.tensor([[1.0,0.0], [0.0,1.0]], dtype=torch.double)
    sigma = torch.tensor([[[0.5249],[0.4072]]], dtype=torch.double) 
    C = torch.tensor([[2.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite
    D = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive definite
    R = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.double)  # Positive semi-definite
    T = torch.tensor(1.0, dtype=torch.double)
    method = 'rk4'
    
    solver = LQRSolver(H, M, sigma, C, D, R, T, method)
    
    N = 5000
    t0 = torch.tensor(0.1,dtype = torch.double)
    
    time_grid_for_MC = torch.linspace(t0,T,N,dtype = torch.double)
    
    X0 = 0.5*torch.ones([1,1,2], dtype=torch.double)
    dt_for_MC = time_grid_for_MC[1:]-time_grid_for_MC[:-1]
    S = solver.solve_riccati_ode(time_grid_for_MC.unsqueeze(0)).squeeze()
    multpX = - M@torch.linalg.inv(D)@M.T
    multpalp = -torch.linalg.inv(D)@M.T@S
    
    params = {
    'R': R,
    'C': C,
    'D': D,
    'X0': X0,
    'H': H,
    'dt': dt_for_MC,
    'multX': multpX,
    'multa':multpalp,
    'S': S,
    'sig': sigma,
    }
    
    iterations = list(range(500))  
    
    with Pool(processes = 4) as pool:  
        J_list = pool.starmap(worker, [(iteration, params) for iteration in iterations])
        
    J_list_tensor = torch.cat(J_list, dim=0)  

Process SpawnPoolWorker-89:
Process SpawnPoolWorker-90:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ysrae1/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/ysrae1/anaconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ysrae1/anaconda3/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/ysrae1/anaconda3/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'worker' on <module '__main__' (built-in)>
  File "/Users/ysrae1/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/ysrae1/anaconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._

In [74]:
import os
os.cpu_count()

12